**Задание #1, Вариант #29**

Исходные данные: $a_{k}$ и $b_{k}$ - задана общая матрица Вандермонда $A$

In [1]:
import numpy as np
from numpy.linalg import eig
from pandas import DataFrame

def a(k):
    return np.divide(k, 3)

def b(k, n):
    return np.power(n + 1 - k, -1, dtype=np.double)

Для начала нам необходим корень матрицы A. Найдем его двумя способами.

Первый способ - через собственные числа и векторы. Не забываем сразу проверить - все ли собственные числа положительные?

In [2]:
def eigenSqrt(A):
    (eigenvalues, eigenvectors) = np.linalg.eig(A)

    if(np.any(eigenvalues <= 0)):  raise TypeError("Negative eigen numbers!")

    sqrt_eigenvalues = np.sqrt(eigenvalues)
    sqrt_eigenvalues_matrix = np.diag(sqrt_eigenvalues)
    
    B = np.dot(np.dot(eigenvectors, sqrt_eigenvalues_matrix), np.linalg.inv(eigenvectors))
    B_square = np.dot(B, B)
    deviation = np.mean(np.abs(B_square - A))
    
    return [np.linalg.cond(B), deviation]

Второй предлагаемый метод из статьи - с помощью метода Ньютона.

In [3]:
def newtonSqrt(A):
    B = [A]

    prev_deviation = None
    deviation = None
    while prev_deviation is None or deviation is None or deviation < prev_deviation:
        if not deviation is None:
            prev_deviation = deviation
        last_B = B[-1]
        if np.linalg.det(last_B) == 0:
            break
        new_B = (last_B + np.dot(np.linalg.inv(last_B), A)) / 2
        deviation = np.mean(np.abs(new_B - last_B))
        B.append(new_B)

    if (len(B) > 2):
        answer_B = B[-2]
    else:
        answer_B = B[-1]
    B_square = np.dot(answer_B, answer_B)
    ans_deviation = np.mean(np.abs(B_square - A))
    return [np.linalg.cond(answer_B), ans_deviation]

Вывод таблицы. **eigen_dev** и **newton_dev** отвечают за погрешность для равенства $A = B^2\$. Через них мы сможем отследить момент наступления неустойчивости процесса.


In [4]:
answer_array = []
for n in range(2,10):
    A = np.array([[a(i + 1) ** b(j + 1, n) for j in range(n)] for i in range(n)])
    (eig_cond_B, eig_dev) = eigenSqrt(A)
    (newton_cond_B, newton_dev) = newtonSqrt(A)
    answer_array.append([n, np.linalg.cond(A), eig_cond_B, newton_cond_B, eig_dev, newton_dev])

df = DataFrame(answer_array, columns=["n", "cond(A)", "cond(B_eig)", "cond(B_newton)", "eig_dev", "newton_dev"])
df

,n,cond(A),cond(B_eig),cond(B_newton),eig_dev,newton_dev
0,2,1.372552e+01,3.805254e+00,3.805254,3.191891e-16,1.526557e-16
1,3,3.906741e+02,1.999373e+01,19.993727,8.696747e-16,1.158541e-10
2,4,2.975096e+04,1.841592e+02,183.387727,3.295975e-16,8.453185e-06
3,5,4.308598e+06,2.464521e+03,347.143670,1.052491e-15,5.885870e-05
4,6,1.000378e+09,4.288602e+04,210.606130,1.179612e-15,1.237788e-03
5,7,3.384474e+11,9.136852e+05,109.993797,1.309610e-15,2.403566e-02
6,8,1.572732e+14,2.301227e+07,32.875206,1.896053e-15,1.040402e-01
7,9,6.897003e+16,3.542439e+08,45.008785,2.135466e-15,1.291907e+00


**Результаты:** как мы можем наблюдать, для обоих предоставленных методов, числа обусловленности для матрицы **B** сначала более-менее равны, однако, начиная с **n = 5** разница увеличивается на порядки. Тем не менее, матрица **A** имеет самые большие числа обусловленности. Различие на порядки видно уже с **n = 2**

При методе через собственные числа и векторы наблюдается отличная погрешность. Чего нельзя сказать о погрешности для метода Ньютона. Погрешность значительно ухудшается уже при **n = 3**, что позволяет сказать о наступлении момента неустойчивости процесса. 

Так же, по причине отсутствия ошибки, делаем вывод о том, что собственные числа матрицы $A$ положительны.